# Representación de las palabras

Como hemos visto en el tema de redes convolucionales, las imágenes se representan como la cantidad de rojo, verde y azul de cada pixel. Esa cantidad es un número que varía de entre 0 y 255. Es decir, para representar una imagen necesitamos hacerlo mediante números.

Con el lenguaje pasa igual, para poder procesarlo y realizar predicciones o generar texto, necesitamos poder representarlo mediante números. Vamos a ver varias formas de representar el lenguaje mediante números: `encoding ordinal`, `one-hot encoding` y `word embedding`

## Encoding ordinal

Esta es la manera más básica de representar numéricamente un lenguaje, y consiste en asignar un número a cada palabra, por ejemplo, podemos decir que gato lo representaremos con un 1, perro con un 2, mesa con un 3, ...

## One-hot encoding

Aunque el encoding ordinal nos resuelve el problema tiene varios problemas

 * En el ejemplo que hemos dado se puede asumir que mesa (3) equivale a gato (1) + perro (2), lo cual no tiene nada que ver. En el lenguaje existen relaciones entre las palabras, por ejemplo perro y perra tienen mucha relación, por lo que es necesario un sistema de codificación mejor
 * Importancia de las palabras. Las palabras van a entrar a redes neuronales, que como hemos visto se componene de capas, con unos pesos, mediante las cuales se realizan unas operaciones matemáticas. Por lo que puede llegar a pasar que la red le de más importancia a las palabras con un número mayor

Debido a esto, se pensó en una alternativa, el one-hot encodding. Aquí lo que se hace es crear vectores de tamaño N, donde cada palabra corresponderá a un vector de todo ceros, menos un uno en una posición determinada. En el ejemplo de antes gato correspondería al vector `[1 0 0 ... 0]`, perro al vector `[0 1 0 ... 0]`, mesa al vector `[0 0 1 ... 0]`, ...

Haciendo esta codificación arreglamos los dos problemas que hemos contado del encoding ordinal y además añadimos una ventaja, como las redes neuronales realizan operaciones matriciales, si a una matriz le multiplicas por un vector de todo ceros, menos un uno en una posición, lo que estás haciendo es que el resultado de la operación es obtener la columna o la fila correspondiente a la posición del 1

![one-hot encodding matrix multiplication](Imagenes/one_hot_encoding_matrix_multiplication.png)

Esto es muy útil si en redes neuronales quieres una fila o una columna de una matriz, porque al no obtener la fila o la columna haciendo slicing, es decir, `matriz[i]`, sino que se obtiene mediante una multiplicación, esta operación es deribable y por tanto se podría añadir al algoritmo del descenso del gradiente

## Word embedding

El one-hot encoding está muy bien, pero crea un nuevo problema, y es que en un lenguaje de un millón de palabras, necesitaríamos que cada palabra se codificase con vectores de 999.999 ceros y 1 solo uno. Esto a la hora de hacer operaciones matriciales no es muy eficiente y hace que las redes y datos ocupen mucha memoria

Para solucionar esto se empezó a usar el hot encoding a secas o también llamado word embedding, que consiste en seguir representando cada palabra en vectores, pero ahora cada vector tendrá un tamaño fijo, de momento digamos N, y en el que cada uno de sus componentes puede tener cualquier valor.

Esto resuelve el problema de la memoria y la ineficiencia en las operaciones matriciales

Además crea una nueva característica, ya que al representar las palabras en vectores de N dimensiones, en realidad lo que estamos haciendo es representar las palabras en un espacio N dimensional. Si esto te suena a muy complicado no te preocupes, que ahora te lo cuento de otra manera , ya verás que sencillo es y cómo lo vas a entender

Supongamos que en vez de ser un espacio de N dimensiones, tenemos un espacio de 2 dimensiones, alto y ancho, pues podríamos representar las palabras de esta manera

![word embedding 2 dimmension](Imagenes/word_embedding_2_dimmension.png)

Como puedes ver todas las palabras que tienen semejanza están juntas. Pues ahora supón que en vez de 2 dimensiones tenemos 3, alto, ancho y profundo, podríamos representar las palabras de esta manera

![word embedding 3 dimmension](Imagenes/word_embedding_3_dimmension.png)

Ahora las palabras siguen estando juntas por semejanza, pero tenemos una dimensión más para poder hacer más grupos.

Como en el lenguaje tenemos muchisimas palabras no nos vale con 3 dimensiones, por lo que tenemos que hacerlo con muchas más. El modelo más grande de word embeding de Bert es de 1024 dimensiones, mientras que el modelo más grande de word embeding de GPT3 es de 4096 dimensiones

Esta posibilidad de poder representar las palabras en grupos por semejanza también nos da otra ventaja, y es la relación entre palabras. Si al vector que representa la palabra `rey` le restas el vector que representa la palabra `hombre` y le sumas el vector que representa la palabra `mujer` obtienes un vector muy parecido al que representa la palabra `reina`.

Como hemos dicho, en el lenguaje existe una gran relación entre las palabras, lo cual va a ser muy importante para entender frases y además es uno de los mecanismos más importantes de los transformers, el de atención, de hecho el paper de los transformers se llama `Attention is all you need` (`Atención es todo lo que necesitas`). Así que gracias a esta forma de representar las palabras podemos prestar atención a esta relación entre palabras

Este tipo de representación de las palabras es como se hace en los transformers, y es el primer bloque de la arquitectura

![word embedding - transformer architecture](Imagenes/transformer_architecture_model_input_embedding.png)

Como hemos explicado, este bloque lo que hace es recibir palabras de un lenguaje y las convierte a vectores de manera que podamos trabajar con ellos

### Como se crean los word embeddings

Hemos explicado cómo se representan muy bien las palabras gracias a los word embedding, hemos mostrado como las palabras con semejanzas están representadas juntas y hemos hablado de los word embeddings de Bert y GPT3. Pero no hemos explicado cómo se crean, es decir, cómo se decide qué valor tiene cada item del vector para cada palabra. Obviamente eso no lo hace ninguna persona

Cuando se entrena un transformer los pesos que están en esta capa se modifican de manera que el transformer consiga el menor `loss` posible. Supongamos el uso inicial que se pensó para los transformers, traducción. Al transformer le entraban palabras, por un lado la capa de `input embedding` generaba vectores con esas palabras y por otro lado el resto de capas hacen su trabajo, que veremos más adelante. Al principio, como los pesos de la capa de `input embedding` son generados aleatoriamente los vectores generados que representan las palabras no tienen ningún sentido. Pero a medida que se va entrenando y se van modificando los pesos mediante el descenso del gradiente, los vectores generados que representan las palabras van teniendo más sentido. Gracias a los mecanismos de atención (que veremos más adelante) las palabras que tienen relación unas con otras se irán posicionando juntas en el espacio vectorial.

### Implementación

Vamos a usar el `input embedding` de Bert para ver cómo representa las palabras, para ello vamos a usar [huggingface](https://huggingface.co) que se hizo muy popular gracias a su librería [transformers](https://huggingface.co/docs/transformers)

Primero vamos a importar las librerías necesarias

In [5]:
from transformers import BertTokenizer, BertModel
import torch

Cargamos el modelo Bert

In [6]:
model = BertModel.from_pretrained('bert-base-multilingual-cased')

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Seguramente obtendrás un warning como yo, ya que estamos importando los pesos de `bert-base-multilingual-cased` que fue entrenado con más capas de las que aparecen en el modelo `BertModel`. Si fuésemos a usar el modelo para lo que fue entrenado y dio como resultado los pesos `bert-base-multilingual-cased` deberíamos importar las capas extra, pero como no es nuestro caso podemos ignorar el warning

Ahora su tokenizador

In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

Creamos el token de la palabra `hola`

In [8]:
input_ids = tokenizer.encode("hola", add_special_tokens=True)

Más adelante entenderás esto, pero el modelo Bert solo contiene la parte de codificación del transformer, por lo que para obtener el embedding de la palabra `hola` lo que hacemos es pasarla por el modelo entero. Primero convertimos su token a un vector y luego hacemos inferencia

In [9]:
input_ids_tensor = torch.tensor([input_ids])
with torch.no_grad():
    outputs = model(input_ids_tensor)

Obtenemos el embedding

In [10]:
# El primer elemento de la salida del modelo son los embeddings para cada token.
# Tomamos el segundo token (índice 1) ya que el primer token es [CLS]
hola_embedding = outputs[0][0][1]

type(hola_embedding), hola_embedding.shape

(torch.Tensor, torch.Size([768]))

Como vemos obtenemos un tensor de tamaño 768, esto quiere decir que el modelo Bert que hemos usado tiene un word embedding de tamaño 768, es decir, cada palabra estará representada por un vector de 768 valores distintos

### Palabras similares en la misma zona del espacio vectorial

Vamos a ver el ejemplo que hemos contado antes, en el que al embedding de la palabra `rey` le restamos el embedding de la palabra `hombre` y le sumamos el embedding de la palabra `mujer` y comparamos el resultado con el embedding de la palabra `reina`. Si son similares es porque como hemos dicho, las palabras similares se encuentran en las mismas zonas del espacio vectorial, por lo que restando y sumando nos llevamos la palabra `rey` a la zona de la palabra `reina`.

Para ver la similitud entre el resultado y el embedding de la palabra reina vamos a utilizar la función `cosine_similarity` que es como se comprueba la similitud entre palabras en un espacio vectorial. Más adelante, cuando expliquemos los bloques de atención explicaremos el por qué de esto. De momento solo quédate, que cuanto más cercano a 1, mas similares serán las palabras

Primero importamos las librerías necesarias

In [11]:
from transformers import BertTokenizer, BertModel
import torch
from torch.nn.functional import cosine_similarity

Ahora cargamos el tockenizador de bert para obtener los embeddings de las palabras

In [12]:
# Carga el pre-trained BERT
model = BertModel.from_pretrained('bert-base-multilingual-cased')
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Como hemos explicado antes puede salir un warning pero no nos afecta

Ahora creamos una lista con las palabras de las que queremos obtener el embedding

In [13]:
# Lista de palabras
palabras = ["rey", "reina", "hombre", "mujer"]

Creamos un dicionario donde vamos a guardar los embedings de las palabras

In [14]:
# Diccionario para guardar los embeddings
embeddings = {}

Obtenemos el embedding de cada palabra

In [15]:
for palabra in palabras:
    # Codifica la palabra en tokens y agrega los tokens especiales [CLS] y [SEP]
    input_ids = tokenizer.encode(palabra, add_special_tokens=True)

    # Convierte los IDs a tensores y pasa por el modelo para obtener los embeddings
    input_ids_tensor = torch.tensor([input_ids])
    with torch.no_grad():
        outputs = model(input_ids_tensor)

    # El primer elemento de la salida del modelo son los embeddings para cada token.
    # Tomamos el segundo token (índice 1) ya que el primer token es [CLS]
    embeddings[palabra] = outputs[0][0][1]

Realizamos la operación de `rey` menos `hombre` más `mujer`

In [16]:
# Realiza la operación rey - hombre + mujer
resultado = embeddings["rey"] - embeddings["hombre"] + embeddings["mujer"]

Y por último calculamos la similitud entre el embedding del cálculo y el embedding de la palabra `reina`

In [17]:
# Calcula la similitud coseno entre el resultado y la palabra "reina"
similitud = cosine_similarity(resultado.unsqueeze(0), embeddings["reina"].unsqueeze(0))
print(f"La similitud coseno entre 'rey - hombre + mujer' y 'reina' es: {similitud.item()}")

La similitud coseno entre 'rey - hombre + mujer' y 'reina' es: 0.728291928768158


Como hemos dicho, ya explicaremos por qué se usa el coseno para calcular la similitud, pero como el resultado es cercano a 1 podemos decir que el embedding resultante de `hombre` - `hombre` + `mujer` es muy similar al embedding de la palabra `reina`

La capa de input embedding del modelo Bert parece que está muy bien conseguida

### Transfer learning

Como hemos explicado, a la hora de entrenar un transformer se entrena también la capa de `input embedding`. Pero en según que casos podemos usar el `input embedding` de un modelo grande de lenguaje, como en este caso el de Bert, o de otros modelos más grnades, cuyo espacio vectorial sea más grande que el que hemos utilizado ahora, o que esté especializado en el dominio en el que queramos trabajar, etc.